# Notebook 5.3: Economic NMPC for Fed-Batch Bioreactor - Maximizing Product

In Notebook 5.2, we designed an NMPC controller to track a glucose setpoint in our fed-batch bioreactor. While useful for maintaining specific conditions, this approach doesn't directly optimize the ultimate process goal, which is often to maximize the amount of product formed.

**Economic Model Predictive Control (EMPC)** changes the paradigm by incorporating a direct economic objective function into the optimization problem. Instead of tracking pre-defined setpoints, the EMPC dynamically determines the optimal operating trajectory (including feed rates) to maximize (or minimize) an economic criterion over the prediction horizon.

For our fed-batch bioreactor, a common economic objective is to maximize the total amount of product at the end of the batch: $P(t_f) \times V(t_f)$.

**Goals of this Notebook:**
1. Understand the concept of Economic MPC (EMPC).
2. Define an economic objective function for the bioreactor: maximizing total final product.
3. Modify the NMPC formulation from Notebook 5.2 (using CasADi) to incorporate this economic objective.
4. Implement the EMPC receding horizon loop, where the prediction horizon typically extends to the anticipated end of the batch.
5. Simulate the closed-loop system and visualize how the EMPC strategy dynamically adjusts the feed rate.
6. Compare the EMPC performance (final product, trajectories) with the setpoint tracking NMPC and open-loop strategies.
7. Discuss the characteristics of EMPC-driven trajectories (e.g., potentially non-constant operating points).

## 1. Importing Libraries and Re-using Code from Notebook 5.1 & 5.2

We'll use the same core libraries and the bioreactor model.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import casadi as ca

# Optional: for nicer plots
plt.rcParams.update({'font.size': 12, 'figure.figsize': (12, 9)}) 

# --- Bioreactor Model ODE Function and Parameters (from Notebook 5.1) ---
default_params = {
    'mu_max': 0.08, 'K_S': 0.1, 'k_d': 0.005, 'k_d_S': 0.02, 'K_d_S_coeff': 0.01,
    'Y_XS': 0.5, 'm_S': 0.01, 'alpha': 0.01, 'beta': 0.002,
    'Y_PS': 1e9, 'S_feed': 200.0
}

def bioreactor_ode_numpy(t, states, params, F_in_val): # For Scipy plant simulation
    Xv, S, P, V = states
    mu_max, K_S, k_d, k_d_S, K_d_S_coeff = params['mu_max'], params['K_S'], params['k_d'], params['k_d_S'], params['K_d_S_coeff']
    Y_XS, m_S, alpha, beta, Y_PS, S_feed = params['Y_XS'], params['m_S'], params['alpha'], params['beta'], params['Y_PS'], params['S_feed']
    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    if Y_PS > 1e6: q_S = (mu / (Y_XS + 1e-9)) + m_S
    else: q_S = (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9))
    if V < 1e-6: return [0,0,0,F_in_val]
    dilution = F_in_val / V
    dXv_dt = (mu - mu_d_val) * Xv - dilution * Xv
    dS_dt = -q_S * Xv + dilution * (S_feed - S)
    dP_dt = q_P * Xv - dilution * P
    dV_dt = F_in_val
    return [dXv_dt, dS_dt, dP_dt, dV_dt]

def bioreactor_ode_casadi(states_sx, params_sx, F_in_sx):
    Xv, S, P, V = states_sx[0], states_sx[1], states_sx[2], states_sx[3]
    mu_max = params_sx['mu_max']; K_S = params_sx['K_S']; k_d = params_sx['k_d']; k_d_S = params_sx['k_d_S']; K_d_S_coeff = params_sx['K_d_S_coeff']
    Y_XS = params_sx['Y_XS']; m_S = params_sx['m_S']; alpha = params_sx['alpha']; beta = params_sx['beta']; Y_PS = params_sx['Y_PS']; S_feed = params_sx['S_feed']
    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    q_S = ca.if_else(Y_PS > 1e6, (mu / (Y_XS + 1e-9)) + m_S, (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9)))
    dilution = F_in_sx / (V + 1e-9)
    dXv_dt = (mu - mu_d_val) * Xv - dilution * Xv
    dS_dt = -q_S * Xv + dilution * (S_feed - S)
    dP_dt = q_P * Xv - dilution * P
    dV_dt = F_in_sx
    return ca.vertcat(dXv_dt, dS_dt, dP_dt, dV_dt)

# Number of states and inputs
nx = 4 # Xv, S, P, V
nu = 1 # F_in

## 2. EMPC Formulation: Maximizing Final Product

**Objective Function for EMPC:**
The primary goal is to maximize the total amount of product at the final predicted time $t_{k+N_p}$. Since optimizers typically minimize, we will minimize the negative of the total product.

Minimize $J_{econ} = - (P_{k+N_p|k} \times V_{k+N_p|k})$

We still need to include penalties on control effort and rate of change to ensure smooth and realistic control actions. These act as regularization terms.

So, the full objective becomes:
Minimize $J = -w_P (P_{k+N_p|k} \cdot V_{k+N_p|k}) + \sum_{j=0}^{N_p-1} R_F (F_{in,k+j|k})^2 + \sum_{j=0}^{N_p-1} S_F (\Delta F_{in,k+j|k})^2$
Where $w_P$ is a weight for the economic term (can be 1 if other terms are scaled appropriately, or used to balance against regularization).

**Prediction Horizon ($N_p$) for EMPC:**
For batch processes, $N_p$ in EMPC is often chosen to cover the **remaining duration of the batch**. At each MPC step $k$, $N_p(k) = (t_{final,batch} - t_k) / T_{s,mpc}$. This means $N_p$ might decrease as the batch progresses. For simplicity in this notebook, we can start with a fixed, sufficiently long $N_p$ that covers most of the expected batch time, or dynamically adjust it. Let's use a fixed long $N_p$ for now.

**Constraints:** The same physical constraints on inputs, states, and volume apply as in Notebook 5.2.

In [6]:
# EMPC Parameters
Ts_empc = 1.0    # hr (EMPC sampling/control interval)
Np_empc = 48     # Prediction horizon (e.g., 48 hours - this might be dynamically adjusted in a real scenario)
                 # This Np should be long enough to see the impact on final product.

# Objective Function Weights for EMPC
w_P_econ = 1.0       # Weight for maximizing product (negative sign in obj)
R_F_econ_weight = 0.01 # Weight for feed rate magnitude (regularization)
S_F_econ_weight = 0.1  # Weight for feed rate change (regularization)
                        # These regularization weights might need to be smaller than in tracking MPC
                        # to allow the economic objective to dominate.

# Constraints (same as Notebook 5.2, can be adjusted)
F_in_min_empc = 0.0
F_in_max_empc = 0.05 
delta_F_in_max_empc = 0.02 # Allow slightly larger changes if needed for optimization

S_min_abs_empc = 0.05
S_max_abs_empc = 20.0 # May allow higher transient S if it helps P
V_max_abs_empc = 2.0 
Xv_min_abs_empc = 0.01

# Final batch time (used if Np is dynamic, or as a reference for fixed Np)
t_batch_final_empc = 240 # hr

## 3. Setting up the EMPC with CasADi

The CasADi setup is very similar to Notebook 5.2, with the main difference being the objective function.

In [7]:
# --- CasADi EMPC Setup ---
opti_empc = ca.Opti()

# Decision variables
X_pred_empc_sym = opti_empc.variable(nx, Np_empc + 1)
U_pred_empc_sym = opti_empc.variable(nu, Np_empc)

# Parameters
x0_empc_param = opti_empc.parameter(nx)
u_prev_empc_param = opti_empc.parameter(nu)
params_num_empc_ode = default_params.copy() # Numerical parameters for the ODE model

# Objective function for EMPC
obj_empc = 0

# Economic term: - (P_final * V_final)
P_final_pred = X_pred_empc_sym[2, Np_empc] # P at k+Np
V_final_pred = X_pred_empc_sym[3, Np_empc] # V at k+Np
obj_empc += -w_P_econ * (P_final_pred * V_final_pred)

# Regularization terms for control inputs
for j in range(Np_empc):
    obj_empc += R_F_econ_weight * (U_pred_empc_sym[0, j])**2
    if j == 0:
        delta_u_econ = U_pred_empc_sym[0, j] - u_prev_empc_param[0]
    else:
        delta_u_econ = U_pred_empc_sym[0, j] - U_pred_empc_sym[0, j-1]
    obj_empc += S_F_econ_weight * delta_u_econ**2

opti_empc.minimize(obj_empc)

# Dynamic constraints (using the same integrator setup as 5.2)
x_cas_e = ca.SX.sym('x_cas_e', nx)
u_cas_e = ca.SX.sym('u_cas_e', nu)
ode_rhs_e = bioreactor_ode_casadi(x_cas_e, params_num_empc_ode, u_cas_e)
ode_func_e = ca.Function('ode_func_e', [x_cas_e, u_cas_e], [ode_rhs_e])

intg_opts_e = {'tf': Ts_empc, 'simplify': True, 'number_of_finite_elements': 4}
dae_e = {'x': x_cas_e, 'p': u_cas_e, 'ode': ode_rhs_e}
intg_e = ca.integrator('intg_e', 'rk', dae_e, intg_opts_e)

opti_empc.subject_to(X_pred_empc_sym[:,0] == x0_empc_param)
for j in range(Np_empc):
    res_e = intg_e(x0=X_pred_empc_sym[:,j], p=U_pred_empc_sym[:,j])
    opti_empc.subject_to(X_pred_empc_sym[:,j+1] == res_e['xf'])

# Boundary and Path Constraints
for j in range(Np_empc):
    opti_empc.subject_to(opti_empc.bounded(F_in_min_empc, U_pred_empc_sym[0,j], F_in_max_empc))
    if j == 0:
        delta_u_constr_e = U_pred_empc_sym[0,j] - u_prev_empc_param[0]
    else:
        delta_u_constr_e = U_pred_empc_sym[0,j] - U_pred_empc_sym[0,j-1]
    opti_empc.subject_to(opti_empc.bounded(-delta_F_in_max_empc, delta_u_constr_e, delta_F_in_max_empc))
    
    opti_empc.subject_to(X_pred_empc_sym[0,j+1] >= Xv_min_abs_empc)  # Xv
    opti_empc.subject_to(opti_empc.bounded(S_min_abs_empc, X_pred_empc_sym[1,j+1], S_max_abs_empc)) # S
    opti_empc.subject_to(X_pred_empc_sym[3,j+1] <= V_max_abs_empc)    # V

# NLP solver options
opts_setting_e = {'ipopt.max_iter': 300, 'ipopt.print_level': 0, 'print_time': 0, 
                  'ipopt.acceptable_tol': 1e-5, 'ipopt.acceptable_obj_change_tol': 1e-5}
opti_empc.solver('ipopt', opts_setting_e)

print("EMPC problem formulated with CasADi.")

EMPC problem formulated with CasADi.


## 4. Implementing the EMPC Receding Horizon Loop

In [8]:
# Simulation Parameters for EMPC run
sim_time_empc_total = t_batch_final_empc # Simulate for the total defined batch time
num_sim_steps_empc = int(sim_time_empc_total / Ts_empc)

# Plant initial conditions
x_plant_empc_current = np.array([0.1, 5.0, 0.0, 1.0]) # [Xv0, S0, P0, V0]
u_plant_empc_prev = np.array([0.0])

# Data logging
t_log_empc = np.zeros(num_sim_steps_empc + 1)
X_log_empc = np.zeros((nx, num_sim_steps_empc + 1))
U_log_empc = np.zeros((nu, num_sim_steps_empc))
Predicted_Pfinal_log = np.zeros(num_sim_steps_empc)

X_log_empc[:, 0] = x_plant_empc_current
t_log_empc[0] = 0

# Initial guess for decision variables
U_guess_empc = np.full((nu, Np_empc), 0.002)
X_guess_empc = np.tile(x_plant_empc_current.reshape(nx,1), (1, Np_empc + 1))

print(f"Starting EMPC simulation for {num_sim_steps_empc} steps...")
for k in range(num_sim_steps_empc):
    current_time = k * Ts_empc
    print(f"EMPC Step {k+1}/{num_sim_steps_empc} (t={current_time:.1f} hr)", end='\r')
    
    # Dynamically adjust Np_empc if desired (for simplicity, keeping it fixed for now)
    # remaining_steps = num_sim_steps_empc - k
    # current_Np = min(Np_empc_fixed, remaining_steps)
    # if current_Np < 1: break # Batch finished
    # If Np changes, the CasADi Opti object needs to be redefined or use parameter for Np.
    # For this notebook, we use fixed Np_empc.

    opti_empc.set_value(x0_empc_param, x_plant_empc_current)
    opti_empc.set_value(u_prev_empc_param, u_plant_empc_prev)
    
    opti_empc.set_initial(X_pred_empc_sym, X_guess_empc)
    opti_empc.set_initial(U_pred_empc_sym, U_guess_empc)
    
    try:
        sol_empc = opti_empc.solve()
        U_optimal_empc = sol_empc.value(U_pred_empc_sym)
        X_predicted_empc = sol_empc.value(X_pred_empc_sym)
        
        u_applied_empc = U_optimal_empc[:, 0]
        Predicted_Pfinal_log[k] = X_predicted_empc[2, -1] * X_predicted_empc[3, -1] # P_final * V_final
        
        X_guess_empc = np.hstack((X_predicted_empc[:, 1:], X_predicted_empc[:, -1].reshape(nx,1)))
        U_guess_empc = np.hstack((U_optimal_empc[:, 1:], U_optimal_empc[:, -1].reshape(nu,1)))
        
    except RuntimeError as e:
        print(f"\nSolver failed at step {k+1} (t={current_time:.1f}hr): {e}")
        print("Using previous control input as fallback.")
        u_applied_empc = u_plant_empc_prev
        Predicted_Pfinal_log[k] = np.nan if k==0 else Predicted_Pfinal_log[k-1]
        U_guess_empc = np.full((nu, Np_empc), u_plant_empc_prev[0])
        X_guess_empc = np.tile(x_plant_empc_current.reshape(nx,1), (1, Np_empc + 1))
        
    U_log_empc[:, k] = u_applied_empc
    
    t_span_plant_e = [current_time, current_time + Ts_empc]
    plant_sol_e = solve_ivp(bioreactor_ode_numpy, 
                            t_span_plant_e, 
                            x_plant_empc_current, 
                            args=(default_params, u_applied_empc[0]),
                            dense_output=False, 
                            t_eval=[current_time + Ts_empc],
                            method='LSODA'
                           )
    x_plant_empc_current = plant_sol_e.y[:, -1]
    x_plant_empc_current = np.maximum(x_plant_empc_current, 0) # Enforce non-negativity
    if x_plant_empc_current[1] < S_min_abs_empc/10 and x_plant_empc_current[0] < Xv_min_abs_empc*5:
        print(f"\nCulture appears depleted or crashed at t={current_time + Ts_empc:.1f} hr. Stopping simulation.")
        # Fill remaining logs if stopping early
        for i_fill in range(k + 1, num_sim_steps_empc):
            U_log_empc[:, i_fill] = u_applied_empc # Hold last input
            Predicted_Pfinal_log[i_fill] = Predicted_Pfinal_log[k]
        for i_fill in range(k + 1, num_sim_steps_empc + 1):
            X_log_empc[:, i_fill] = x_plant_empc_current
            t_log_empc[i_fill] = (i_fill) * Ts_empc
        # Adjust num_sim_steps_empc to current k for plotting
        num_sim_steps_empc = k + 1 
        break 

    X_log_empc[:, k + 1] = x_plant_empc_current
    t_log_empc[k + 1] = current_time + Ts_empc
    u_plant_empc_prev = u_applied_empc

print("\nEMPC simulation finished.")

Starting EMPC simulation for 240 steps...


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## 5. Visualizing EMPC Performance

In [ ]:
# Adjust time vectors if simulation stopped early
t_log_empc_plot = t_log_empc[:num_sim_steps_empc+1]
X_log_empc_plot = X_log_empc[:, :num_sim_steps_empc+1]
U_log_empc_plot = U_log_empc[:, :num_sim_steps_empc]
Predicted_Pfinal_log_plot = Predicted_Pfinal_log[:num_sim_steps_empc]
time_for_u_plot = t_log_empc_plot[:-1]

fig_e, axs_e = plt.subplots(5, 1, figsize=(12, 18), sharex=True)
fig_e.suptitle(f'EMPC for Bioreactor Product Maximization (Np={Np_empc}, wP={w_P_econ}, Rf={R_F_econ_weight}, Sf={S_F_econ_weight})', fontsize=16)

# Glucose (S)
axs_e[0].plot(t_log_empc_plot, X_log_empc_plot[1, :], 'b-', label='$S_{actual}$ (g/L)')
axs_e[0].axhline(S_min_abs_empc, color='m', linestyle='--', label='$S_{min}$ Constraint')
axs_e[0].axhline(S_max_abs_empc, color='m', linestyle='--', label='$S_{max}$ Constraint')
axs_e[0].set_ylabel('Glucose S (g/L)'); axs_e[0].grid(True); axs_e[0].legend()
axs_e[0].set_ylim([-0.1, S_max_abs_empc + 1])

# Viable Cell Density (Xv) and Product (P)
ax1e_twin = axs_e[1].twinx()
axs_e[1].plot(t_log_empc_plot, X_log_empc_plot[0, :], 'g-', label='$X_v$ (g/L)')
ax1e_twin.plot(t_log_empc_plot, X_log_empc_plot[2, :], 'c--', label='$P_{prod}$ (g/L)')
axs_e[1].set_ylabel('$X_v$ (g/L)', color='g'); ax1e_twin.set_ylabel('$P_{prod}$ (g/L)', color='c')
axs_e[1].tick_params(axis='y', labelcolor='g'); ax1e_twin.tick_params(axis='y', labelcolor='c')
axs_e[1].grid(True); axs_e[1].legend(loc='upper left'); ax1e_twin.legend(loc='upper right')

# Feed Rate (F_in)
axs_e[2].step(time_for_u_plot, U_log_empc_plot[0, :], 'k-', where='post', label='$F_{in}$ (L/hr)')
axs_e[2].axhline(F_in_max_empc, color='m', linestyle='--', label='$F_{in,max}$ Constraint')
axs_e[2].set_ylabel('Feed Rate $F_{in}$ (L/hr)'); axs_e[2].grid(True); axs_e[2].legend()
axs_e[2].set_ylim([-0.005, F_in_max_empc + 0.005])

# Volume (V)
axs_e[3].plot(t_log_empc_plot, X_log_empc_plot[3, :], 'purple', label='Volume V (L)')
axs_e[3].axhline(V_max_abs_empc, color='m', linestyle='--', label='$V_{max}$ Constraint')
axs_e[3].set_ylabel('Volume V (L)'); axs_e[3].grid(True); axs_e[3].legend()
axs_e[3].set_ylim([0, V_max_abs_empc + 0.2])

# Predicted Final Total Product at each step k
axs_e[4].plot(time_for_u_plot, Predicted_Pfinal_log_plot, 'orange', label='Predicted $P_{final}V_{final}$ at step k')
axs_e[4].set_ylabel('Predicted Total P (g)')
axs_e[4].set_xlabel('Time (hr)'); axs_e[4].grid(True); axs_e[4].legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

final_total_product_empc = X_log_empc_plot[2, -1] * X_log_empc_plot[3, -1]
print(f"Final conditions at t={t_log_empc_plot[-1]:.1f} hr (EMPC):")
print(f"  Xv = {X_log_empc_plot[0, -1]:.3f} g/L")
print(f"  S  = {X_log_empc_plot[1, -1]:.3f} g/L")
print(f"  P  = {X_log_empc_plot[2, -1]:.3f} g/L")
print(f"  V  = {X_log_empc_plot[3, -1]:.3f} L")
print(f"  Total Product = {final_total_product_empc:.3f} g")

## 6. Discussion and Comparison

*   **EMPC Trajectories:** Compare the glucose profile and feed rate profile generated by the EMPC with those from the setpoint-tracking NMPC (Notebook 5.2) and a simple open-loop strategy (e.g., constant feed from Notebook 5.1). Are they different? How?
    *   EMPC might choose to operate glucose at levels that are not constant if that leads to better overall productivity. For example, it might allow glucose to drop lower during peak production if cells are more productive under slight limitation, or keep it higher during growth phase.
*   **Final Product:** How does the final total product achieved by EMPC compare to the other strategies? (You would need to run comparable simulations from Notebook 5.1 and 5.2 for a similar batch duration).
*   **Role of Regularization ($R_F, S_F$):** These weights are still important in EMPC. If they are too small, the optimizer might choose very aggressive or rapidly changing feed rates that are unrealistic or detrimental in practice (even if the model suggests they are optimal for the economic term). They help to shape a more implementable policy.
*   **Prediction Horizon $N_p$:** For EMPC of batch processes, $N_p$ ideally represents the remaining batch time. If $N_p$ is too short, the EMPC might make greedy short-term decisions that are not globally optimal for the entire batch.
*   **Computational Cost:** EMPC problems, especially with long horizons, can be more computationally demanding than tracking NMPC with shorter horizons.

**Exercises:**
1.  Adjust the economic weight `w_P_econ` and the regularization weights `R_F_econ_weight`, `S_F_econ_weight`. How do these changes affect the feed strategy and the final product?
2.  Try different (fixed) `Np_empc` values. How does a shorter vs. longer horizon impact the results?
3.  Modify the economic objective. For example, include a cost for the total feed used: Minimize $-w_P (P_{final}V_{final}) + w_{FeedCost} \int F_{in}(t) dt$. How does this change the strategy?
4.  Run a simulation from Notebook 5.2 (tracking NMPC, e.g., tracking $S_{glc}=2.0$) for the same total batch time (`sim_time_empc_total`) and compare the final total product with the EMPC result.

## 7. Key Takeaways

*   Economic MPC directly optimizes an economic objective function, rather than tracking pre-defined setpoints for individual variables.
*   For batch processes like fed-batch bioreactors, EMPC can determine dynamic operating strategies (e.g., feed profiles) that maximize overall economic performance (e.g., final product yield).
*   The prediction horizon in batch EMPC often corresponds to the remaining batch duration to allow for long-term optimization.
*   Regularization terms (penalties on control effort/rate) are still important in EMPC to ensure practical and smooth control actions.
*   EMPC can lead to different, and often superior, operating strategies compared to traditional setpoint-tracking MPC when the true goal is economic optimization.

This notebook demonstrates the power of shifting the control objective towards direct economic optimization. The next optional notebooks could explore state estimation for these bioreactor NMPC/EMPC controllers or introduce concepts for perfusion bioreactors.